In [55]:
import numpy as np
from stl import mesh
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pyro
import pyro.distributions as dist
import pyro.contrib.examples.util  # patches torchvision
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam


In [56]:
def calculate_angle(x,y,z):
         return np.arccos(np.sum((x-y)*(x-z)))/(np.linalg.norm(x-y)*np.linalg.norm(x-z))
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3


In [62]:
###TODO: fix numerical instability
def createRIMD(reference,deformed):
    print('getting reference')
    #gets point and implicit neighbouroods of reference
    your_mesh0 = mesh.Mesh.from_file(reference)
    temp0=your_mesh0.data[0][1]
    for i in range(1,len(your_mesh0.data)):
        temp0=np.concatenate((temp0,your_mesh0.data[i][1]), axis=0)
    Pointset0 = np.unique(temp0, axis=0)
    neigh0=[]
    N=len(Pointset0)
    for i in range(len(Pointset0)):
        lst=[]
        for j in range(len(your_mesh0.data)):
            if Pointset0[i].tolist() in your_mesh0.data[j][1].tolist():
                for k in range(3):
                    if Pointset0[i].tolist()!=your_mesh0.data[j][1][k].tolist():
                        lst.append(your_mesh0.data[j][1][k].tolist())
        neigh0.append(lst)
    #gets point and and neigh of deformed
    print('getting deformed')
    your_mesh1 = mesh.Mesh.from_file(deformed)
    temp1=your_mesh1.data[0][1]
    for i in range(1,len(your_mesh1.data)):
        temp1=np.concatenate((temp1,your_mesh1.data[i][1]), axis=0)
    Pointset1 = np.unique(temp1, axis=0)
    neigh1=[]
    for i in range(N):
        lst=[]
        for j in range(len(your_mesh1.data)):
            if Pointset1[i].tolist() in your_mesh1.data[j][1].tolist():
                for k in range(3):
                    if Pointset1[i].tolist()!=your_mesh1.data[j][1][k].tolist():
                        lst.append(your_mesh1.data[j][1][k].tolist())
        neigh1.append(lst)
    print('calculating C')
     #calculates the matrix of the angles
    C=np.zeros([N,N])
    for i in range(N):
        for j in range(N):
            if i!=j:
                temp=intersection(neigh0[i],neigh0[j])
                if len(temp)==2:            
                    C[i,j]=abs(1/np.tan(calculate_angle(temp[0],Pointset0[i],Pointset0[j])))+abs(1/np.tan(calculate_angle(temp[1],Pointset0[i],Pointset0[j])))
     #calculates real neigh
    print('calculating real neigh')
    neigh=[1]*N
    for i in range(N):
        neigh[i]=[j for j in range(N) if C[i,j]!=0]
    print('calculating T')
    #calculates T
    T=np.zeros([N,3,3])
    for i in range(N):
        for a in range(3):
            for b in range(3):
                temp0=0
                temp1=0
                for j in neigh[i]: 
                        p0=Pointset0[i]-Pointset0[j]
                        p1=Pointset1[i]-Pointset1[j]
                        temp0=temp0+C[i,j]*p0[b]*p1[a]
                        temp1=temp1+C[i,j]*p0[b]*p0[b]
                T[i,a,b]=temp0/temp1
    T=np.nan_to_num(T,nan=1)
    R=np.zeros([N,3,3])
    S=np.zeros([N,3,3])
    dR=np.zeros([N,N,3,3])
    temp=np.zeros([N,3,3])
    print('calculating R and S')
    #calculates R and S
    for i in range(N):
        temp[i]=np.matmul(T[i].T,T[i])
        D,Q=np.linalg.eigh(temp[i])
        D=np.diag(D)**0.5
        D=np.nan_to_num(D,nan=1)
        S[i]=np.matmul(Q.T,np.matmul(D,Q))
        R[i]=np.matmul(T[i],np.linalg.inv(S[i]))
        R[i][R[i]==np.inf] = 999

    print('calculating RIMD')
    RIMDi0=S[0]
    for j in neigh[0]:
        dR[0][j]=np.matmul(R[0].T,R[j])
        RIMDi0=np.concatenate((dR[0,j],RIMDi0),axis=1)
    RIMD=RIMDi0
    for i in range(N):
        RIMDi=S[i]
        for j in neigh[i]:
            dR[i][j]=np.matmul(R[i].T,R[j])
            RIMDi=np.concatenate((dR[i,j],RIMDi),axis=1)
        RIMD=np.concatenate((RIMD,RIMDi),axis=1)
    M=len(RIMD[0,:])
    RIMD=RIMD.reshape(3*M)
    return RIMD

In [63]:
RIMD=createRIMD('orig.stl','front0.stl')
RIMDtilde=2*0.9*(RIMD-np.min(RIMD))/(np.max(RIMD)-np.min(RIMD))-0.9

getting reference
getting deformed
calculating C
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
calculating real neigh
calculating T


ZeroDivisionError: division by zero

In [51]:
class Decoder(nn.Module):
    def __init__(self, z_dim, hidden_dim):
        super().__init__()
        # setup the two linear transformations used
        self.fc1 = nn.Linear(z_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, 3*70236)
        # setup the non-linearities
        self.softplus = nn.Softplus()
        self.sigmoid = nn.Sigmoid()

    def forward(self, z):
        # define the forward computation on the latent z
        # first compute the hidden units
        hidden = self.softplus(self.fc1(z))
        # return the parameter for the output Bernoulli
        # each is of size batch_size x 784
        loc_img = self.sigmoid(self.fc21(hidden))
        return loc_img

class Encoder(nn.Module):
    def __init__(self, z_dim, hidden_dim):
        super().__init__()
        # setup the three linear transformations used
        self.fc1 = nn.Linear(3*70236, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, z_dim)
        self.fc22 = nn.Linear(hidden_dim, z_dim)
        # setup the non-linearities
        self.softplus = nn.Softplus()

    def forward(self, x):
        # define the forward computation on the image x
        x = x.reshape(3*70236)
        # then compute the hidden units
        hidden = self.softplus(self.fc1(x))
        # then return a mean vector and a (positive) square root covariance
        # each of size batch_size x z_dim
        z_loc = self.fc21(hidden)
        z_scale = torch.exp(self.fc22(hidden))
        return z_loc, z_scale

        
class VAE(nn.Module):
    # by default our latent space is 50-dimensional
    # and we use 400 hidden units
    def __init__(self, z_dim=50, hidden_dim=400, use_cuda=False):
        super().__init__()
        # create the encoder and decoder networks
        self.encoder = Encoder(z_dim, hidden_dim)
        self.decoder = Decoder(z_dim, hidden_dim)

        if use_cuda:
            # calling cuda() here will put all the parameters of
            # the encoder and decoder networks into gpu memory
            self.cuda()
        self.use_cuda = use_cuda
        self.z_dim = z_dim

    # define the model p(x|z)p(z)
    def model(self, x):
        # register PyTorch module `decoder` with Pyro
        pyro.module("decoder", self.decoder)
        with pyro.plate("data", x.shape[0]):
            # setup hyperparameters for prior p(z)
            z_loc = x.new_zeros(torch.Size((x.shape[0], self.z_dim)))
            z_scale = x.new_ones(torch.Size((x.shape[0], self.z_dim)))
            # sample from prior (value will be sampled by guide when computing the ELBO)
            z = pyro.sample("latent", dist.Normal(z_loc, z_scale).to_event(1))
            # decode the latent code z
            loc_img = self.decoder(z)
            # score against actual images
            pyro.sample("obs", dist.Bernoulli(loc_img).to_event(1), obs=x.reshape(-1, 3*70236))

    # define the guide (i.e. variational distribution) q(z|x)
    def guide(self, x):
        # register PyTorch module `encoder` with Pyro
        pyro.module("encoder", self.encoder)
        with pyro.plate("data", x.shape[0]):
            # use the encoder to get the parameters used to define q(z|x)
            z_loc, z_scale = self.encoder(x)
            # sample the latent code z
            pyro.sample("latent", dist.Normal(z_loc, z_scale).to_event(1))
            
    def fitted(self, x):
        # encode image x
        z_loc, z_scale = self.encoder(x)
        # sample in latent space
        z = dist.Normal(z_loc, z_scale).sample()
        # decode the image (note we don't sample in image space)
        loc_img = self.decoder(z)
        return loc_img

def train(svi, train_loader, use_cuda=False):
    # initialize loss accumulator
    epoch_loss = 0.
    # do a training epoch over each mini-batch x returned
    # by the data loader
    for x in train_loader:
        # if on GPU put mini-batch into CUDA memory
        if use_cuda:
            x = x.cuda()
        # do ELBO gradient and accumulate loss
        epoch_loss += svi.step(x)
        
    # return epoch loss
    normalizer_train = len(train_loader.dataset)
    total_epoch_loss_train = epoch_loss / normalizer_train
    return total_epoch_loss_train

def evaluate(svi, test_loader, use_cuda=False):

    # initialize loss accumulator
    test_loss = 0.
    # compute the loss over the entire test set
    for x in test_loader:
        # if on GPU put mini-batch into CUDA memory
        if use_cuda:
            x = x.cuda()
        # compute ELBO estimate and accumulate loss
        test_loss += svi.evaluate_loss(x)

        hello=vae.fitted(x)
        
    return hello



In [46]:
# Run options
LEARNING_RATE = 1.0e-3
USE_CUDA = False
# Run only for a single iteration for testing
NUM_EPOCHS = 100 
TEST_FREQUENCY = 10

train_loader=[]
train_loader.append(torch.tensor(RIMDtilde).float())
print(RIMDtilde)
# clear param store
pyro.clear_param_store()

# setup the VAE
vae = VAE(use_cuda=USE_CUDA)

# setup the optimizer
adam_args = {"lr": LEARNING_RATE}
optimizer = Adam(adam_args)

# setup the inference algorithm
svi = SVI(vae.model, vae.guide, optimizer, loss=Trace_ELBO())

train_elbo = []
test_elbo = []
# training loop
for epoch in range(NUM_EPOCHS):
    total_epoch_loss_train = train(svi, train_loader, use_cuda=USE_CUDA)
    train_elbo.append(-total_epoch_loss_train)
    print("[epoch %03d]  average training loss: %.4f" % (epoch, total_epoch_loss_train))
'''
    if epoch % TEST_FREQUENCY == 0:
        # report test diagnostics
        total_epoch_loss_test = evaluate(svi, test_loader, use_cuda=USE_CUDA)
        test_elbo.append(-total_epoch_loss_test)
        print("[epoch %03d] average test loss: %.4f" % (epoch, total_epoch_loss_test))
'''


[-0.41010871 -0.4101087  -0.41010871 ... -0.41010871 -0.41010871
 -0.41010871]


RuntimeError: [enforce fail at alloc_cpu.cpp:73] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 177591445056 bytes. Error code 12 (Cannot allocate memory)
        Trace Shapes:                 
         Param Sites:                 
 decoder$$$fc1.weight    400     50   
   decoder$$$fc1.bias           400   
decoder$$$fc21.weight 210708    400   
  decoder$$$fc21.bias        210708   
        Sample Sites:                 
            data dist             |   
                value 210708      |   
          latent dist 210708      | 50
                value 210708      | 50

6